# Trucks only problem #

Install necessary packages

In [2]:
from gurobipy import Model,GRB,LinExpr,quicksum
import numpy as np
from scipy.spatial import distance
import os
from load_dataset import Dataset

Define model parametres

In [3]:
## MODEL PARAMETERS ##
W_T = 1500 #empty weight truck [kg]
Q_T = 1000 #load capacity of trucks [kg]
W_D = 25 #empty weight drone [kg]
Q_D = 5 #load capacity of drones [kg]
C_T = 25 #travel cost of trucks per unit distance [monetary unit/km]
C_D = 1 #travel cost of drones per unit distance [monetary unit/km]
C_B = 500 #basis cost of using a truck equipped with a drone [monetary unit]
E = 0.5 #maximum endurance of empty drones [hours]
S_T = 60 #average travel speed of the trucks [km/h]
S_D = 65 #average travel speed of the drones [km/h]

Define Big M constant

In [4]:
M = 500 #big M constant for big M method

Load Dataset using load_dataset.py

In [5]:
## LOAD DATASET ##
current_dir = os.getcwd()
# Select which data folder to use
data_subfolder = '0.3'
data_num_nodes = '40'
data_area = '20'

data_file_name = f'{data_num_nodes}_{data_area}_{data_subfolder}'
dataset_path = f'dataset/{data_subfolder}/{data_file_name}.txt'
output_file_path = os.path.join(current_dir, data_file_name + '_solution.sol')#used to save solution file

dataset = Dataset(dataset_path)

Pre-processing

In [6]:
## FUNCTIONS ##
def get_manhattan_distance(data):
    """
    Returns a dictionary with manhattan distances between all nodes in dataset
    """
    distance_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            distance_dict[node1, node2] = distance.cityblock([data[node1]['X'], data[node1]['Y']], [data[node2]['X'], data[node2]['Y']])
    return distance_dict

def get_time_dict(data, S_T, distance_dict):
    """
    Returns a dictionary with travel times between all nodes in dataset
    """
    time_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            time_dict[node1, node2] = distance_dict[node1, node2] / S_T
    return time_dict


num_trucks = 2
distance_dict = get_manhattan_distance(dataset.data)
time_dict = get_time_dict(dataset.data, S_T, distance_dict)

#definitions of N_0, N and N_plus follow from paper
N = list(dataset.data.keys()) #set of nodes with depot at start
N_customers = N.copy()
N_customers.remove('D0')
V = [f'V{i}' for i in range(1, num_trucks+1)] #set of trucks

Create the model

In [7]:
## MODEL ##
model = Model("Truck-Only Model")

#decision variables
#define x such that you cannot travel between same node
x = model.addVars(V, [(i,j) for i in N for j in N if i != j], lb=0, ub=1, vtype=GRB.BINARY, name='x')
y = model.addVars(V, lb=0, ub=1, vtype=GRB.BINARY, name='y')
t = model.addVars(V, N, lb=0, vtype=GRB.CONTINUOUS, name='t')

model.update()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2518537
Academic license 2518537 - for non-commercial use only - registered to j.___@student.tudelft.nl


Define the constraints

In [8]:
#constraints
# C1: each customer is visited exactly once
C1 = model.addConstrs((quicksum(x[v,i,j] for j in N if i != j for v in V) == 1 for i in N_customers), name='C1') 

# C2: each truck leaves the depot exactly once if it is active (y=1)
C2 = model.addConstrs((quicksum(x[v,'D0',j] for j in N if 'D0' != j) - y[v] == 0 for v in V), name='C2')

# C3: each truck arrives at depot once if it is active
C3 = model.addConstrs((quicksum(x[v,i,'D0'] for i in N if i != 'D0') - y[v] == 0 for v in V), name='C3')

# C4: if vehicle arrives at customer, must also leave
C4 = model.addConstrs((quicksum(x[v,i,h] for i in N if i != h) - quicksum(x[v,h,j] for j in N if h != j) == 0 for h in N_customers for v in V), name='C4')

# C5: time constraint (time at node equal or larger than time at previous node plus travel time)
C5 = model.addConstrs((t[v,j] >= t[v,i] + time_dict[i,j] - M*(1-x[v,i,j]) for i in N for j in N if i != j for v in V), name='C5')

# C6: payload for all visited customer nodes per vehicle less than limit Q_T
C6 = model.addConstrs((dataset.data[i]['Demand'] * quicksum(x[v,i,j] for j in N if i != j) <= Q_T for i in N_customers for v in V), name='C6')
# At least one truck must be active (otherwise optimal solution is to use no trucks)
#C7 = model.addConstr(quicksum(y[v] for v in V) >= 0.99, name='C7')
#C9 = model.addConstrs((x[v,i,i] == 0 for i in N for v in V), name='C9')#note: this constraint is essential (ensures you cant travek between same node) otherwise truck never leaves depot

Run the optimiser

In [9]:
#objective function (minimise cost both due to tranportation and basis cost of using truck (if active, i.e. y=1))
cost_obj = quicksum(C_T * distance_dict[i,j] * x[v,i,j] for i in N for j in N if i != j for v in V) + quicksum(C_B * y[v] for v in V)
model.setObjective(cost_obj, GRB.MINIMIZE)
model.update()
model.write('TruckonlySimple.lp')
#tune solver before optimizing to reduce time it takes
#model.tune()
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2518537 - for non-commercial use only - registered to j.___@student.tudelft.nl
Optimize a model with 3484 rows, 3364 columns and 22804 nonzeros
Model fingerprint: 0xa8311b38
Variable types: 82 continuous, 3282 integer (3282 binary)
Coefficient statistics:
  Matrix range     [6e-01, 5e+02]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 80 rows and 0 columns
Presolve time: 0.03s
Presolved: 3404 rows, 3364 columns, 19604 nonzeros
Variable types: 82 continuous, 3282 integer (3282 binary)

Root relaxation: objective 5.590000e+03, 173 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Jonathan van Zyl\.conda\envs\gurobi_env\lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 11638  8496 6330.33483   37   56          - 6315.00000      -  12.7   25s
 18556 14799 6527.18586  183   68          - 6315.00000      -  11.5   30s
 25819 20697 6762.73983   83   31          - 6315.00000      -  11.4   35s
 31517 25523 6465.29050   58   62          - 6315.00000      -  11.6   40s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Jonathan van Zyl\.conda\envs\gurobi_env\lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 35808 29134 9248.19883  130   28          - 6315.00000      -  11.7   50s
 37488 31376 6320.19735   33   61          - 6315.00000      -  11.6   64s
 39977 32645 11359.8858  194   16          - 6315.00000      -  11.6   65s
 48331 39475 6493.18983   32   59          - 6315.00000      -  11.6   70s


Post-processing

In [ ]:
## POST-PROCESSING ##
solution = {}
for var in model.getVars():
    solution[var.varName] = var.x

#exctract active vehicles
active_vehicles = [v for v in V if solution[f'y[{v}]'] >= 0.99]
#extract routes
active_routes = {}
for v in active_vehicles:
    active_routes[v] = [i for i in N if solution[f'x[{v},{i},D0]'] >= 0.99] #start with depot
    while active_routes[v][-1] != 'D0':
        for j in N:
            if solution[f'x[{v},{active_routes[v][-1]},{j}]'] >= 0.99:
                active_routes[v].append(j)
                break

#retrieve timestamps of customer visits
timestamps = {}
for v in active_vehicles:
    timestamps[v] = {}
    for i in N_customers:
        timestamps[v][i] = solution[f't[{v},{i}]']

#print all solution variables which have value of 1
print([var for var in solution.keys() if solution[var] >=0.9])
print(active_routes)
#plot routes
dataset.plot_data(show_demand=True, scale_nodes=True, show_labels=False, active_routes=active_routes)